# Сеть Коско

Согласно [1] нейронная сеть Коско, или двунаправлленная ассоциативная память
(Biderectional associative memory, BAM) — тип рекуррентой нейронной сети,
предложеный Бартом Коско в [2].

Интерпретация двунаправленной ассоциативной памяти с помощью нейронных сетей —
двухслойная иерархия симметрично соединённых нейронов. Когда нейроны
активируются, сеть быстро переходит в стабильное состояние реверберации с двумя
паттернами. Стабильная ревербеация соответствует локальному минимуму энергии
[2]. При обучении или использовании адаптивной сети Коско стабильная
реверберация паттерна $\left(A_i, B_i\right)$ помещает информацию в
долгосрочную память соединения $M_i$.

Данная нейронная сеть состоит из двух слоёв [3]: количество $K$ нейронов
первого слоя равно количеству бинарных входных элементов, а количество нейронов
 $M$ выходного слоя равно числу выходных элементов.

Значения входных и выходных элементов принадлежат множеству $\{-1; 1\}$.

Сеть Коско решает задачу класификации и позволяет определить класс входных
данных из известных ей классов либо дать заключение, что вход не принадлежит ни
одному из известных  классов.

Данная нейронная сеть задаётся матрицей весов $W$ [5]. Входной вектор
обрабатывается матрицей весов $W$, в результате и подаётся на вход функции
активации $F$, в результате получаем выходной вектор $B=F\left(AW\right)$.
Затем выход рекуррентно подаётся на вход сети: $A=F\left(BW^T\right)$. Данная
 операция повторяется до стабилизации входных и выходных значений.

Ассоциации хранятся в матрице весов $W$.

Если имеются обучающие выборки ассоциаций $A$ и $B$, то матрица весовых коэффициентов $W$ может быть найдена как сумма произведений:

$$W=\sum_i A_i^T B_i$$

In [1]:
import numpy as np
import random
from sklearn.datasets import fetch_openml
from sklearn.model_selection import train_test_split
import typing

Реализация [5]

In [2]:
class BAM:
    K: int
    M: int
    w: np.ndarray

    def __init__(self, M: int, K: int):
        """

        :param M: число выходных нейронов
        :param K: число входных нейронов
        """
        self.M = M
        self.K = K

    def predict(self, A: np.ndarray, depth=3, mode='encrypt') -> np.ndarray:
        A1 = A
        B1 = self.activation(A1.dot(self.w if mode == 'encrypt' else self.w.T))

        while True:
            # print(A1.shape)
            # print(B1.shape)
            A2 = self.activation(B1.dot(self.w.T if mode == 'encrypt' else self.w))
            B2 = self.activation(A2.dot(self.w if mode == 'encrypt' else self.w.T))
            if depth <= 0 or ((A1 == A2).all() and (B1 == B2).all()):
                return np.vectorize(self.bipolar)(B2)
            A1, B1 = A2, B2
            depth -= 1

    def activation(self, X: np.ndarray, l=10.0) -> np.ndarray:
        return 2/(1+np.exp(-l*X))-1

    def fit(self, A: list[np.ndarray], B:list[np.ndarray]) -> typing.NoReturn:
        self.w = np.zeros((self.M, self.K))

        for i in range(len(A)):
            # print(A[i].shape)
            # print(B[i].shape)
            k = A[i].T.dot(B[i])
            print(k)
            # print(self.w.shape)
            self.w += k
            # if i > 1:
            #     break

    @staticmethod
    def bipolar(x: float) -> int:
        return 1 if x>0 else -1

## Примеры использования

### 1 Шифр подстановки

Допустим необходимо закодировать сообщение "00011011" используя сследующую таблицу подстановки:

00 -> 000
01 -> 101
10 -> 110
11 -> 001

In [3]:
[BAM.bipolar(int(i)) for i in "000"]

[-1, -1, -1]

In [4]:
X_train = [np.array([[BAM.bipolar(int(j)) for j in i]]) for i in ["00","01","10", "11"]]
y_train = [np.array([[BAM.bipolar(int(j)) for j in i]]) for i in ["000","011","101", "110"]]

In [5]:
bam = BAM(2, 3)
bam.fit(X_train, y_train)

[[1 1 1]
 [1 1 1]]
[[ 1 -1 -1]
 [-1  1  1]]
[[ 1 -1  1]
 [-1  1 -1]]
[[ 1  1 -1]
 [ 1  1 -1]]


In [6]:
def get_accuracy(nn: BAM, X:list[np.ndarray], y:list[np.ndarray], depth=3, /, verbose=True, mode = 'encrypt'):
    MSE = 0
    for i in range(len(y)):
        yi = nn.predict(X[i], depth, mode=mode)
        # print(y[i],yi)
        try:
            if (yi-y[i]).all:
                MSE +=  1
            elif verbose:
                print(X[i], yi, y[i])
        except:
            if yi==y[i]:
                MSE +=  1
    return (MSE / len(y))


# def get_MSE(nn, X, y, depth=3):
#     MSE = 0.0
#     for i in range(len(y)):
#         yi = nn.predict(X[i], depth)[0]
#         k: np.ndarray = yi - y[i][0]
#         MSE += k.sum()**2
#     return (MSE / len(y))**.5

In [7]:
print("train")
for i in range(1,):
    print(get_accuracy(bam, X_train, y_train, i))

train
1.0


попробуем закодировать, а затем декодировать сообщение

In [8]:
message = [np.array([[BAM.bipolar(int(j)) for j in i]]) for i in ["00","01","10", "11", "10"]]

In [9]:
ciphred = [bam.predict(np.array(i), 3) for i in message]
ciphred

[array([[-1, -1, -1]]),
 array([[-1,  1, -1]]),
 array([[ 1, -1, -1]]),
 array([[ 1,  1, -1]]),
 array([[ 1, -1, -1]])]

In [10]:
deciphred = [bam.predict(i, 3, mode='decrypt') for i in ciphred]
print(deciphred)
n_err = 0
for i in range(len(message)):
    print(message[i].shape, deciphred[i].shape)
    n_err += abs(message[i][0,0]-deciphred[i][0,0])
    n_err += abs(message[i][0,1]-deciphred[i][0,1])
print(n_err)

[array([[-1, -1]]), array([[-1,  1]]), array([[ 1, -1]]), array([[1, 1]]), array([[ 1, -1]])]
(1, 2) (1, 2)
(1, 2) (1, 2)
(1, 2) (1, 2)
(1, 2) (1, 2)
(1, 2) (1, 2)
0


### 2

Использование сети Коско для кодирования/декодирования зашумлённых сигналов.

В работе [4] рассматривается использование сетей Коско для шифрования данных без пересылки ключа.

Допустим необходимо переслать строку

In [50]:
text_to_transfer = "A similar operation is to be applied on the receiver side as well, the integer value is to be fed to the inverse g(x) function and the value thus obtained is used to define the target values, than dot operation between the targets formed and transform of key provides us with the required input data. It is converted from binary to ASCII values and finally to the corresponding character. Hence we successfully created encrypted data and later decrypted it at receiver side itself, without transferring the data itself. The algorithm can be summarized as follows:"

Договоримся на сторонах о функции `g(s)`, например:

In [51]:
def g(s):
    return 2*s**2+3*s+5

**На стороне отправителя** необходимо выполнить следующие операции:

Выберем число бит шифрованого текста, коорым хотим кодировать один бит исходного текста, например

In [52]:
s = len(text_to_transfer)

t = s
while t < 9:# Возможно только если s==2
    t+=s
t

563

Сформируем единичную матрицу порядка `t`:

In [53]:
out = np.eye(t).reshape(t,t,1)

переведём сообщение в двоичный вид, использую s бит на символ

In [54]:
input_msg: list[int] = []
for ii in text_to_transfer:
    input_msg += [int(i) for i in bin(ord(ii))[2:].zfill(s)]

In [55]:
col = len(input_msg) // t
input_message: np.ndarray = np.array(input_msg).reshape((t, col,1))

Динамически генерируем ключ

In [56]:
num = np.random.randint(1,5)
out = np.roll(out, num, 0)
print(num)

1


Рассчитаем матрицу весовых коэффициентов

In [57]:
weight = np.zeros((col, t))

for i in range(t):
    k = np.dot(input_message[i], out[i].T)
    weight += k

Передаём получателю ключ `g(t)`, `num` и матрицу `weight`.

На этапе разработки прогаммы значение `t` легко получить из `w.shape`, однако при передаче по сети матрица передаётся в линейном виде последовательностью t*col символов, так как результат - состовное число, которое может иметь несколько разложений на множители.

Чтобы найти `t`, необходимо знать функцию `g(t)`.

In [58]:
key1=g(t)
key2=g(num)
key3=weight

***


На стороне получателя:

1. Получим `t` из `key1` и `num` из `key2`

In [59]:
D1=(9-8*(2-key1))**.5

t1 = (-3-D1)/4
t2 = (-3+D1)/4

t = int(t1 if t1>0 and t1.is_integer() else t2)


D2=(9-8*(2-key2))**.5

num1 = (-3-D2)/4
num2 = (-3+D2)/4

num = int(num1 if num1>0 and num1.is_integer() else num2)

In [60]:
print(t,num)

563 1


In [61]:
out_r: np.ndarray = np.eye(t).reshape(t, t, 1)
out_r = np.roll(out_r, num, 0)

Расшифровываем сообщение:

In [64]:

msg = []

for i in range(t):
    k = np.dot(weight, out_r[i]).T
    msg.append([int(i) for i in k.tolist()[0]])

Декодируем байты в символы:

In [67]:
k = []
for i in msg:
    n = 0
    for j in i:
        n = 2 * n + j
    k.append(chr(n))
print(''.join(k))

'A similar operation is to be applied on the receiver side as well, the integer value is to be fed to the inverse g(x) function and the value thus obtained is used to define the target values, than dot operation between the targets formed and transform of key provides us with the required input data. It is converted from binary to ASCII values and finally to the corresponding character. Hence we successfully created encrypted data and later decrypted it at receiver side itself, without transferring the data itself. The algorithm can be summarized as follows:'

В результате получаем исходное сообщение.

В данном примере кодируется весь текст, однако на практике возможно применение разделение текста на сегменты переменной длины, что затруднит декодирование текста третьими лицами.

Также сам текст сообщения не передаётся, а передаётся лишь матрица весов нейронной сети.

## Литература

1. Нейронная сеть Коско // Википедия. 2019. URL: https://ru.wikipedia.org/w/index.php?title=%D0%9D%D0%B5%D0%B9%D1%80%D0%BE%D0%BD%D0%BD%D0%B0%D1%8F_%D1%81%D0%B5%D1%82%D1%8C_%D0%9A%D0%BE%D1%81%D0%BA%D0%BE&oldid=99902385 (дата доступа: 05.07.2021).

2. Kosko B. Bidirectional associative memories // IEEE Trans. Syst., Man, Cybern. 1988. Vol. 18, № 1. P. 49–60. URL: http://ieeexplore.ieee.org/document/87054/ (accessed: 05.07.2021).

3. Гетероассоциативная память // Искусственный интеллект, искусственный разум, системы искусственного интеллекта [Электронный ресурс]. URL: https://neuronus.com/theory/nn/961-geteroassotsiativnaya-pamyat.html (дата обращения: 05.07.2021).

4. Pandey K. Bidirectional Associative Memory Neural Network for Data Encryption and Decryption // International Journal for Research in Applied Science and Engineering Technology. 2018. Vol. 6. P. 1744–1750. doi:10.22214/ijraset.2018.6257.

5. Яхъяева Г.Э. Нечеткие множества и нейронные сети: учеб. пособие. Москва: Интернет-Университет Информ. Технологий БИНОМ. Лаб. знаний, 2006. 314 p. URL: https://intuit.ru/studies/courses/88/88/info (дата обращения: 08.07.2021).